In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
mnist_X = np.r_[mnist.train.images, mnist.test.images]
mnist_y = np.r_[mnist.train.labels, mnist.test.labels]

train_X, test_X, train_y, test_y = train_test_split(mnist_X, mnist_y, test_size=0.2, random_state=42)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
import tensorflow.contrib.slim as slim

In [133]:
class Normal(object):
    def __init__(self, loc, scale, givens=[], outputs=[], seed=1, name=None, *kwargs):
        self._set_distribution(loc, scale)
        self.loc = loc
        self.givens = givens
        self.outputs = outputs
        self.seed = seed
        
        self.loc = loc
        self.scale = scale
        self.graph = self.loc.graph
        
        self.name = name
        
    def _set_distribution(self, loc, scale):
        self.distribution = tf.contrib.distributions.Normal(loc=loc, scale=scale)

    def fprop(self, givens=None, **kwargs): 
        if givens == None:
            return self.loc, self.scale
        else:
            loc,scale, = tf.import_graph_def(self.loc.graph.as_graph_def(), input_map=givens,
                                           return_elements=[self.loc.name, self.scale.name], name=self.name)
            self._set_distribution(loc, scale)
            return loc, scale

    def sample_given_x(self, givens=None, **kwargs):
        _ = self.fprop(givens)
        samples = self.distribution.sample()
        return {self.outputs[0]: samples}
    
    def log_likelihood_given_x(self, givens, samples, **kwargs):
        _= self.fprop(givens)
        return self.distribution.log_prob(samples)

In [134]:
class Bernoulli(object):
    def __init__(self, logits, givens=[], outputs=[], seed=1, name=None, *kwargs):
        self._set_distribution(logits)
        self.logits = logits
        self.givens = givens
        self.outputs = outputs
        self.seed = seed
        
        self.logits = logits
        self.graph = self.logits.graph
        
        self.name = name

    def _set_distribution(self, logits):
        self.distribution = tf.contrib.distributions.Bernoulli(logits=logits)        
        
    def fprop(self, givens=None, **kwargs): 
        if givens == None:
            return self.logits
        else:
            logits, = tf.import_graph_def(self.logits.graph.as_graph_def(), input_map=givens,
                                               return_elements=[self.logits.name], name=self.name)
            self._set_distribution(logits)
            return logits

    def sample_given_x(self, givens=None, **kwargs):
        _ = self.fprop(givens)
        samples = self.distribution.sample()
        return {self.outputs[0]: samples}
    
    def log_likelihood_given_x(self, givens, samples, **kwargs):
        _= self.fprop(givens)
        return self.distribution.log_prob(samples)    

In [140]:
with tf.Graph().as_default() as q_graph:
    x = tf.placeholder(tf.float32, shape=[None, 28*28], name="x")
    with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.relu):
        q_0 = slim.fully_connected(x, 64)
        q_1 = slim.fully_connected(q_0, 64)
        q_mean = slim.fully_connected(q_1, 64, activation_fn=None)
        q_var = slim.fully_connected(q_1, 64, activation_fn=tf.nn.tanh)

with tf.Graph().as_default() as p_graph:        
    z = tf.placeholder(tf.float32, shape=[None, 64], name="z")
    with slim.arg_scope([slim.fully_connected], activation_fn=tf.nn.relu):
        p_0 = slim.fully_connected(z, 64)
        p_1 = slim.fully_connected(p_0, 64)
        p_mean = slim.fully_connected(p_1, 784, activation_fn=tf.nn.sigmoid)
        
q = Normal(q_mean, q_var, givens=["x"], outputs=["z"])
p = Bernoulli(p_mean, givens=["z"], outputs=["x"])

In [137]:
_x = tf.placeholder(tf.float32, shape=[None, 28*28], name="x")
z = q.sample_given_x({"x":_x})
#loglike = p.log_likelihood_given_x(z, _x)

In [115]:
print(tf.get_default_graph().get_operations())

[<tf.Operation 'x' type=Placeholder>, <tf.Operation 'import/x' type=Placeholder>, <tf.Operation 'import/fully_connected/weights/Initializer/random_uniform/shape' type=Const>, <tf.Operation 'import/fully_connected/weights/Initializer/random_uniform/min' type=Const>, <tf.Operation 'import/fully_connected/weights/Initializer/random_uniform/max' type=Const>, <tf.Operation 'import/fully_connected/weights/Initializer/random_uniform/RandomUniform' type=RandomUniform>, <tf.Operation 'import/fully_connected/weights/Initializer/random_uniform/sub' type=Sub>, <tf.Operation 'import/fully_connected/weights/Initializer/random_uniform/mul' type=Mul>, <tf.Operation 'import/fully_connected/weights/Initializer/random_uniform' type=Add>, <tf.Operation 'import/fully_connected/weights' type=VariableV2>, <tf.Operation 'import/fully_connected/weights/Assign' type=Assign>, <tf.Operation 'import/fully_connected/weights/read' type=Identity>, <tf.Operation 'import/fully_connected/biases/Initializer/Const' type=C

In [141]:
import tensorboard as tb
tb.show_graph(tf.get_default_graph())
#tb.show_graph(q_graph)

In [139]:
tf.reset_default_graph()

In [146]:
with tf.Session(graph=q_graph) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(q_var, feed_dict={x:train_X[:10]}).shape)

(10, 64)


In [131]:
with tf.Session() as sess:
    _x = tf.placeholder(tf.float32, shape=[None, 28*28], name="x")
    sess.run(tf.global_variables_initializer())
    print(sess.run(q.sample_given_x({"x":_x})["z"], feed_dict={_x:train_X[:10]}).shape)

FailedPreconditionError: Attempting to use uninitialized value import/fully_connected/weights
	 [[Node: import/fully_connected/weights/read = Identity[T=DT_FLOAT, _class=["loc:@import/fully_connected/weights"], _device="/job:localhost/replica:0/task:0/gpu:0"](import/fully_connected/weights)]]
	 [[Node: Normal/sample/Reshape/_3 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_80_Normal/sample/Reshape", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'import/fully_connected/weights/read', defined at:
  File "/home/masa/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/masa/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/masa/.pyenv/versions/anaconda3-4.0.0/lib/pyth

NameError: name 'import_5' is not defined